In [11]:
#----------------------------------------------------------
# Notebook to create database "immobilier" from excel file
#
# Upgrades :
# => code SQLalchemy class to automate table creation
# => automate key generation between tables
#----------------------------------------------------------

from My_db_connector import My_db_connector
import sqlite3 as sql
import pandas as pd
import numpy as np

# load data and drop duplicates
df = pd.read_excel("immobilier.xlsx")
df = df.drop_duplicates()

# formatting attributs
df.columns = [i.replace(' ', '_').lower() for i in df.columns]
df

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,date_mutation,nature_mutation,valeur_fonciere,no_voie,b/t/q,code_type_de_voie,type_de_voie,code_voie,voie,code_id_commune,...,section,no_plan,no_volume,1er_lot,surface_carrez_du_1er_lot,nombre_de_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales
0,2020-02-03,Vente,56000.0,190.0,A,0,RUE,5,CENTRALE,0,...,B,1478,NaN,22,50.42,1,2,Appartement,52,2
1,2020-01-02,Vente,165000.0,347.0,NaN,0,RUE,20,DU CHATEAU,1,...,A,302,NaN,12,48.22,2,2,Appartement,48,3
2,2020-01-08,Vente,720000.0,58.0,NaN,1,AV,527,DU MONT BLANC,2,...,AK,563,NaN,146,130.80,2,2,Appartement,130,6
3,2020-01-06,Vente,429250.0,140.0,NaN,0,RUE,2,DE L'ABBE JOLIVET,3,...,C,2307,NaN,11,109.22,1,1,Maison,109,5
4,2020-01-07,Vente,220900.0,39.0,NaN,0,RUE,110,BUFFON,4,...,AE,440,NaN,31,108.65,2,2,Appartement,91,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34164,2020-06-30,Vente,1290000.0,4.0,NaN,0,RUE,8249,DE RIVOLI,3213,...,AN,3,NaN,113,102.09,1,2,Appartement,99,3
34165,2020-06-30,Vente,726000.0,38.0,NaN,0,RUE,9191,DU TEMPLE,3213,...,AG,79,NaN,67,55.95,1,2,Appartement,56,3
34166,2020-06-25,Vente,420000.0,13.0,NaN,0,RUE,9382,DES TOURNELLES,3213,...,AO,73,NaN,22,32.86,2,2,Appartement,29,2
34167,2020-06-25,Vente,725000.0,16.0,NaN,0,RUE,1797,CHARLEMAGNE,3213,...,AM,67,NaN,89,56.80,1,2,Appartement,48,3


In [2]:
# dataframe summary
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34169 entries, 0 to 34168
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date_mutation              34169 non-null  datetime64[ns]
 1   nature_mutation            34169 non-null  object        
 2   valeur_fonciere            34151 non-null  float64       
 3   no_voie                    34036 non-null  float64       
 4   b/t/q                      2174 non-null   object        
 5   code_type_de_voie          34169 non-null  int64         
 6   type_de_voie               33229 non-null  object        
 7   code_voie                  34169 non-null  object        
 8   voie                       34169 non-null  object        
 9   code_id_commune            34169 non-null  int64         
 10  code_postal                34168 non-null  float64       
 11  commune                    34169 non-null  object        
 12  code

In [4]:
# select all attributs needed in logement_table from df.index
logement_attr = list(df.columns[[3, 6, 7, 8, 11]])
print("table 'logement' :", "\n", logement_attr, "\n")

# select all attributs needed in vente_table from df.index
vente_attr = list(df.columns[[0, 2, 18, 23, 21, 9, 11, 12]])
print("table 'vente' :", "\n", vente_attr)

table 'logement' : 
 ['no_voie', 'type_de_voie', 'code_voie', 'voie', 'commune'] 

table 'vente' : 
 ['date_mutation', 'valeur_fonciere', 'surface_carrez_du_1er_lot', 'nombre_pieces_principales', 'type_local', 'code_id_commune', 'commune', 'code_departement']


In [5]:
# create logement dataframe and drop duplicates to have unique "logements" in this table
df_logement = df.loc[:, logement_attr]
df_logement = df_logement.drop_duplicates().reset_index()
df_logement

,index,no_voie,type_de_voie,code_voie,voie,commune
0,0,190.0,RUE,5,CENTRALE,SAINT-ETIENNE-DU-BOIS
1,1,347.0,RUE,20,DU CHATEAU,CHEVRY
2,2,58.0,AV,527,DU MONT BLANC,DIVONNE-LES-BAINS
3,3,140.0,RUE,2,DE L'ABBE JOLIVET,PERON
4,4,39.0,RUE,110,BUFFON,VALSERHONE
...,...,...,...,...,...,...
30691,34163,6.0,RUE,1284,DE BRETONVILLIERS,PARIS 04
30692,34164,4.0,RUE,8249,DE RIVOLI,PARIS 04
30693,34165,38.0,RUE,9191,DU TEMPLE,PARIS 04
30694,34166,13.0,RUE,9382,DES TOURNELLES,PARIS 04


In [6]:
# create vente dataframe and add vente_id

df_vente = df.loc[:, vente_attr]

# add foreign_key logement_id to df_vente
for i in range(len(df_vente)):
    result, = np.where(df_logement["commune"] == df_vente["commune"].loc[i])
    df_vente.loc[i, "logement_id"] = result[0]
del i, result

df_vente

,date_mutation,valeur_fonciere,surface_carrez_du_1er_lot,nombre_pieces_principales,type_local,code_id_commune,commune,code_departement,logement_id
0,2020-02-03,56000.0,50.42,2,Appartement,0,SAINT-ETIENNE-DU-BOIS,1,0.0
1,2020-01-02,165000.0,48.22,3,Appartement,1,CHEVRY,1,1.0
2,2020-01-08,720000.0,130.80,6,Appartement,2,DIVONNE-LES-BAINS,1,2.0
3,2020-01-06,429250.0,109.22,5,Maison,3,PERON,1,3.0
4,2020-01-07,220900.0,108.65,4,Appartement,4,VALSERHONE,1,4.0
...,...,...,...,...,...,...,...,...,...
34164,2020-06-30,1290000.0,102.09,3,Appartement,3213,PARIS 04,75,30478.0
34165,2020-06-30,726000.0,55.95,3,Appartement,3213,PARIS 04,75,30478.0
34166,2020-06-25,420000.0,32.86,2,Appartement,3213,PARIS 04,75,30478.0
34167,2020-06-25,725000.0,56.80,3,Appartement,3213,PARIS 04,75,30478.0


In [7]:
# add vente_id and logement_id to vente_attr
df_vente["vente_id"] = df_vente.index
vente_attr.append("vente_id")
vente_attr.append("logement_id")

# add logement_id to logement_attr
df_logement["logement_id"] = df_logement.index
logement_attr.append("logement_id")

print("table 'vente' :", "\n", vente_attr, "\n")
print("table 'logement' :", "\n", logement_attr)

table 'vente' : 
 ['date_mutation', 'valeur_fonciere', 'surface_carrez_du_1er_lot', 'nombre_pieces_principales', 'type_local', 'code_id_commune', 'commune', 'code_departement', 'vente_id', 'logement_id'] 

table 'logement' : 
 ['no_voie', 'type_de_voie', 'code_voie', 'voie', 'commune', 'logement_id']


In [8]:
# connection to database
connector = My_db_connector("immobilier")

# populate both tables with to_sql() method
connector.df_to_sql(df_vente, "vente")
connector.df_to_sql(df_logement, "logement")

Connection to 'immobilier' database successful


30696

In [9]:
# display all table from database
tables = ["vente", "logement"]
query = "SELECT * FROM {}"

for table in tables:
    result = connector.db_execute_query(query.format(table))
    display(result)
    
print("Database 'immobilier' ready to run")

,date_mutation,valeur_fonciere,surface_carrez_du_1er_lot,nombre_pieces_principales,type_local,code_id_commune,commune,code_departement,logement_id,vente_id
0,2020-02-03 00:00:00,56000.0,50.42,2,Appartement,0,SAINT-ETIENNE-DU-BOIS,1,0.0,0
1,2020-01-02 00:00:00,165000.0,48.22,3,Appartement,1,CHEVRY,1,1.0,1
2,2020-01-08 00:00:00,720000.0,130.80,6,Appartement,2,DIVONNE-LES-BAINS,1,2.0,2
3,2020-01-06 00:00:00,429250.0,109.22,5,Maison,3,PERON,1,3.0,3
4,2020-01-07 00:00:00,220900.0,108.65,4,Appartement,4,VALSERHONE,1,4.0,4
...,...,...,...,...,...,...,...,...,...,...
34164,2020-06-30 00:00:00,1290000.0,102.09,3,Appartement,3213,PARIS 04,75,30478.0,34164
34165,2020-06-30 00:00:00,726000.0,55.95,3,Appartement,3213,PARIS 04,75,30478.0,34165
34166,2020-06-25 00:00:00,420000.0,32.86,2,Appartement,3213,PARIS 04,75,30478.0,34166
34167,2020-06-25 00:00:00,725000.0,56.80,3,Appartement,3213,PARIS 04,75,30478.0,34167


,index,no_voie,type_de_voie,code_voie,voie,commune,logement_id
0,0,190.0,RUE,5,CENTRALE,SAINT-ETIENNE-DU-BOIS,0
1,1,347.0,RUE,20,DU CHATEAU,CHEVRY,1
2,2,58.0,AV,527,DU MONT BLANC,DIVONNE-LES-BAINS,2
3,3,140.0,RUE,2,DE L'ABBE JOLIVET,PERON,3
4,4,39.0,RUE,110,BUFFON,VALSERHONE,4
...,...,...,...,...,...,...,...
30691,34163,6.0,RUE,1284,DE BRETONVILLIERS,PARIS 04,30691
30692,34164,4.0,RUE,8249,DE RIVOLI,PARIS 04,30692
30693,34165,38.0,RUE,9191,DU TEMPLE,PARIS 04,30693
30694,34166,13.0,RUE,9382,DES TOURNELLES,PARIS 04,30694


Database 'immobilier' ready to run


In [10]:
# close connection 
connector.db_close_connection()

"Connection to 'immobilier' database closed successfully"